# curses — Terminal handling for character-cell displays

https://docs.python.org/3/library/curses.html

Please copy the code in a ```.py``` file and run ```python file.py```. Do not name the file ```curses.py``` as it would give you error.

### only for windows

```
pip install windows-curses
```

In [ ]:
# Uses the curses library to create 3 randomly sized windows with different color
# backgrounds on the screen.

# Note that this is not the most efficient way to code this, but I want to break out
# the individual objects so that it is easier to trace what is going on.

import curses
import math
import random
import sys

# A set of layouts, to be randomly chosen.
layouts = ['2 top, 1 bottom', '2 left, 1 right',
          '1 top, 2 bottom', '1 left, 2 right']


def main(argv):
    # Initialize the curses object.
    stdscr = curses.initscr()

    # Clear screen
    
    stdscr.clear()
    
    # refresh screen
    # stdscr.refresh()
    
    # Do not echo keys back to the client.
    curses.noecho()

    # Non-blocking or cbreak mode... do not wait for Enter key to be pressed.
    curses.cbreak()

    # Turn off blinking cursor
    curses.curs_set(False)

    # Enable color if we can...
    if curses.has_colors():
        curses.start_color()

    # Optional - Enable the keypad. This also decodes multi-byte key sequences
    # stdscr.keypad(True)

    # Beginning of Program...
    # Create a list of all the colors except for black and white. These will server as
    # the background colors for the windows. Because these constants are defined in
    # ncurses,
    # we can't create the list until after the curses.initscr call:
    bgColors = [curses.COLOR_BLUE, curses.COLOR_CYAN, curses.COLOR_GREEN,
                curses.COLOR_MAGENTA, curses.COLOR_RED, curses.COLOR_YELLOW]
    colors = random.sample(bgColors, 3)

    # Create 3 ncurses color pair objects.
    curses.init_pair(1, curses.COLOR_WHITE, colors[0])
    curses.init_pair(2, curses.COLOR_WHITE, colors[1])
    curses.init_pair(3, curses.COLOR_WHITE, colors[2])

    caughtExceptions = ""

    try:
        # Note that print statements do not work when using ncurses. If you want to write
        # to the terminal outside of a window, use the stdscr.addstr method and specify
        # where the text will go. Then use the stdscr.refresh method to refresh the
        # display.
        # stdscr.addstr(0, 0, "Gonna make some windows.")
        # stdscr.refresh()

        # The lists below will eventually hold 4 values, the X and Y coordinates of the
        # top-left corner relative to the screen itself, and the number of characters
        # going right and down, respectively.
        # window1 = []
        # window2 = []
        # window3 = []

        # The variables below will eventually contain the window objects.
        window1Obj = ""
        window2Obj = ""
        window3Obj = ""

        # The variables below will correspond roughly to the X, Y coordinates of the
        # of each window.
        window1 = []
        window2 = []
        window3 = []

        # There's going to be a caption at the bottom left of the screen, but it needs to
        # go in the proper window.
        window1Caption = ""
        window2Caption = ""
        window3Caption = ""

        # The randomly sized windows that don't take up one side of the screen shouldn't
        # be less than 1/3 the screen size, or more than one third of the screen size on
        # either edge.
        minWindowWidth = math.floor(curses.COLS * 1.0/3.0)
        maxWindowWidth = math.floor(curses.COLS * 2.0/3.0)
        minWindowHeight = math.floor(curses.LINES * 1.0/3.0)
        maxWindowHeight = math.floor(curses.LINES * 2.0/3.0)

        # Pick a layout. The random.randrange command will return a value between 0 and 3.
        chosenLayout = layouts[random.randrange(0, 4)]

        if '2 top, 1 bottom' == chosenLayout:
            # Windows 1 and 2 will be the top, Window 3 will be the bottom.
            window1Width = random.randrange(minWindowWidth, maxWindowWidth)
            window1Height = random.randrange(minWindowHeight, maxWindowHeight)
            window1 = [0, 0, window1Width, window1Height]

            window2Width = curses.COLS - window1Width
            window2Height = window1Height
            window2 = [window1Width, 0, window2Width, window2Height]

            window3 = [0, window1Height, curses.COLS,
                      curses.LINES - window1Height]
            window3Caption = chosenLayout + " - Press a key to quit."

        elif '2 left, 1 right' == chosenLayout:
            # Windows 1 and 2 will be on the left, Window 3 will be on the right.
            window1Width = random.randrange(minWindowWidth, maxWindowWidth)
            window1Height = random.randrange(minWindowHeight, maxWindowHeight)
            window1 = [0, 0, window1Width, window1Height]

            window2Width = window1Width
            window2Height = curses.LINES - window1Height
            window2 = [0, window1Height, window2Width, window2Height]
            window2Caption = chosenLayout + " - Press a key to quit."

            window3Width = curses.COLS - window1Width
            window3Height = curses.LINES
            window3 = [window1Width, 0, window3Width, window3Height]

        elif '1 top, 2 bottom' == chosenLayout:
            # Window 1 will be on the top, Windows 2 and 3 will be on the bottom.
            window1Width = curses.COLS
            window1Height = random.randrange(minWindowHeight, maxWindowHeight)
            window1 = [0, 0, window1Width, window1Height]

            window2Width = random.randrange(minWindowWidth, maxWindowWidth)
            window2Height = curses.LINES - window1Height
            window2 = [0, window1Height, window2Width, window2Height]
            window2Caption = chosenLayout + " - Press a key to quit."

            window3Width = curses.COLS - window2Width
            window3Height = window2Height
            window3 = [window2Width, window1Height,
                      window3Width, window3Height]

        elif '1 left, 2 right' == chosenLayout:
            # Window 1 will be on the left, Windows 2 and 3 will be on the right.
            window1Width = random.randrange(minWindowWidth, maxWindowWidth)
            window1Height = curses.LINES
            window1 = [0, 0, window1Width, window1Height]
            window1Caption = chosenLayout + " - Press a key to quit."

            window2Width = curses.COLS - window1Width
            window2Height = random.randrange(minWindowHeight, maxWindowHeight)
            window2 = [window1Width, 0, window2Width, window2Height]

            window3Width = window2Width
            window3Height = curses.LINES - window2Height
            window3 = [window1Width, window2Height,
                      window3Width, window3Height]

        # Create and refresh each window. Put the caption 2 lines up from bottom
        # in case it wraps. Putting it on the last line with no room to wrap (if
        # the window is too narrow for the text) will cause an exception.

        # The newwin() function creates a new window of a given size, returning the new window object.
        
        window1Obj = curses.newwin(
            window1[3], window1[2], window1[1], window1[0])
        
        # window.bkgd(ch[, attr])
        # Set the background property of the window to the character ch, with attributes attr. The change
        # is then applied to every character position in that window:
        #   * The attribute of every character in the window is changed to the new background attribute.
        #   * Wherever the former background character appears, it is changed to the new background character.
        window1Obj.bkgd(' ', curses.color_pair(1))
        
        # Calculate rough center...
        window1Center = [math.floor(window1[2]/2.0),
                        math.floor(window1[3]/2.0)]
        # Add the string to the center, with BOLD flavoring.
        window1Obj.addstr(window1Center[1], window1Center[0] - 4, "Window 1",
                          curses.color_pair(1) | curses.A_BOLD)
        if "" != window1Caption:
            window1Obj.addstr(curses.LINES - 2, 0, window1Caption,
                              curses.color_pair(1) | curses.A_BOLD)
        window1Obj.refresh()

        window2Obj = curses.newwin(
            window2[3], window2[2], window2[1], window2[0])
        window2Obj.bkgd(' ', curses.color_pair(2))
        # Calculate rough center...
        window2Center = [math.floor(window2[2]/2.0),
                        math.floor(window2[3]/2.0)]
        # Add the string to the center, with BOLD flavoring.
        window2Obj.addstr(window2Center[1], window2Center[0] - 4, "Window 2",
                          curses.color_pair(2) | curses.A_BOLD)
        if "" != window2Caption:
            # The "Y coordinate" here is the bottom of the *window* and not the screen.
            window2Obj.addstr(window2[3] - 2, 0, window2Caption,
                              curses.color_pair(2) | curses.A_BOLD)
        window2Obj.refresh()

        window3Obj = curses.newwin(
            window3[3], window3[2], window3[1], window3[0])
        window3Obj.bkgd(' ', curses.color_pair(3))
        # Calculate rough center...
        window3Center = [math.floor(window3[2]/2.0),
                        math.floor(window3[3]/2.0)]
        # Add the string to the center, with BOLD flavoring.
        window3Obj.addstr(window3Center[1], window3Center[0] - 4, "Window 3",
                          curses.color_pair(3) | curses.A_BOLD)
        if "" != window3Caption:
            # The "Y coordinate" here is the bottom of the *window* and not the screen.
            window3Obj.addstr(window3[3] - 2, 0, window3Caption,
                              curses.color_pair(3) | curses.A_BOLD)
        window3Obj.refresh()

        # Necessary so we can "pause" on the window output before quitting.
        window3Obj.getch()

        # Debugging output.
        # stdscr.addstr(0, 0, "Chosen layout is [" + chosenLayout + "]")
        # stdscr.addstr(1, 10, "Window 1 params are [" + str (window1)+ "]")
        # stdscr.addstr(2, 10, "Window 2 params are [" + str(window2) + "]")
        # stdscr.addstr(3, 10, "Window 3 params are [" + str(window3)+ "]")
        # stdscr.addstr(4, 10, "Colors are [" + str(colors) + "]")
        # stdscr.addstr(5, 0, "Press a key to continue.")
        # stdscr.refresh()
        # stdscr.getch()
    except Exception as err:
        caughtExceptions = str(err)

    # End of Program...
    # Turn off cbreak mode...
    curses.nocbreak()

    # Turn echo back on.
    curses.echo()

    # Restore cursor blinking.
    curses.curs_set(True)

    # Turn off the keypad...
    # stdscr.keypad(False)

    # Restore Terminal to original state.
    curses.endwin()

    # Display Errors if any happened:
    if "" != caughtExceptions:
        print("Got error(s) [" + caughtExceptions + "]")
    return 0


if __name__ == "__main__":
    main(sys.argv[1:])


## Examples from Github

## 1. Printing text in center of screen

### `stdscr.getmaxyx()`

Returns a tuple `(y, x)` of the height and width of the window.

Let us assume we have to print a text `Hello, World!` in center of the screen. Then, coordinates will be:

```python
h, w = stdscr.getmaxyx()

x = w//2 - len(text)//2
y = h//2
```

![](images/3.png)


## 2. Setting background and foreground color

### `curses.has_colors()`

Returns True if the terminal can display colors; otherwise, returns False.


### `curses.init_pair(pair_number, fg, bg)`

Create/update a color pair scheme for a given `pair_number`.


### `curses.color_pair(color_number)`

Returns the **attribute value** for displaying text in the specified color.


### `stdscr.attron(attr)`

Adds attribute `attr` to the “background” set applied to all writes to the current window.


### `stdscr.attroff(attr)`

Removes attribute `attr` from the “background” set applied to all writes to the current window.


Now, in order to print text with red color and yellow background,

```python

# create color pair scheme
curses.init_pair(1, curses.COLOR_RED, curses.COLOR_YELLOW)

# activate the color pair scheme
curses.attron(curses.color_pair(1))

# write something
curses.addstr(0, 0, "Hello, World!")

# deactivate the color pair scheme
curses.attroff(curses.color_pair(1))
```

## Example 3

```python
import time
import curses

def main(stdscr):
	# turn off cursor blinking
	curses.curs_set(0)

	# get height and width of screen
	h, w = stdscr.getmaxyx()

	# create a new color scheme
	curses.init_pair(1, curses.COLOR_RED, curses.COLOR_YELLOW)

	# text to be written in center
	text = "Hello, world!"

	# find coordinates for centered text
	x = w//2 - len(text)//2
	y = h//2

	# set color scheme
	stdscr.attron(curses.color_pair(1))

	# write text on screen
	stdscr.addstr(y, x, text)

	# unset color scheme
	stdscr.attroff(curses.color_pair(1))

	# update the screen
	stdscr.refresh()

	# wait for 3 sec before exit
	time.sleep(3)


curses.wrapper(main)
```

## 3. Taking user input

### `stdscr.getch()`

Get an input character.


## 4. Detecting Special Keys

For special keys like `Up`, `Down`, `Left`, `Right`, etc, `curses` has special values. You can get them like this:

- `curses.KEY_UP`
- `curses.KEY_DOWN`
- `curses.KEY_RIGHT`
- `curses.KEY_DOWN`

and so on.


Let's write a simple program to take **arrow keys** and **enter key** as input from user.

## Example 4

```python
import curses

def main(stdscr):

	while 1:
		key = stdscr.getch()

		# clear existing texts
		stdscr.clear()

		if key == curses.KEY_UP:
			stdscr.addstr(0, 0, "You pressed Up key!")
		elif key == curses.KEY_DOWN:
			stdscr.addstr(0, 0, "You pressed Down key!")
		elif key == curses.KEY_ENTER or key in [10, 13]:
			stdscr.addstr(0, 0, "You pressed Enter.")
		else:
			break

		# update screen
		stdscr.refresh()


curses.wrapper(main)
```

Having learnt all the necessary concepts, lets make a simple **Menu Display** application now.

## Example 5

```python
import curses

menu = ['Home', 'Play', 'Scoreboard', 'Exit']


def print_menu(stdscr, selected_row_idx):
	stdscr.clear()
	h, w = stdscr.getmaxyx()
	for idx, row in enumerate(menu):
		x = w//2 - len(row)//2
		y = h//2 - len(menu)//2 + idx
		if idx == selected_row_idx:
			stdscr.attron(curses.color_pair(1))
			stdscr.addstr(y, x, row)
			stdscr.attroff(curses.color_pair(1))
		else:
			stdscr.addstr(y, x, row)
	stdscr.refresh()


def print_center(stdscr, text):
	stdscr.clear()
	h, w = stdscr.getmaxyx()
	x = w//2 - len(text)//2
	y = h//2
	stdscr.addstr(y, x, text)
	stdscr.refresh()


def main(stdscr):
	# turn off cursor blinking
	curses.curs_set(0)

	# color scheme for selected row
	curses.init_pair(1, curses.COLOR_BLACK, curses.COLOR_WHITE)

	# specify the current selected row
	current_row = 0

	# print the menu
	print_menu(stdscr, current_row)

	while 1:
		key = stdscr.getch()

		if key == curses.KEY_UP and current_row > 0:
			current_row -= 1
		elif key == curses.KEY_DOWN and current_row < len(menu)-1:
			current_row += 1
		elif key == curses.KEY_ENTER or key in [10, 13]:
			print_center(stdscr, "You selected '{}'".format(menu[current_row]))
			stdscr.getch()
			# if user selected last row, exit the program
			if current_row == len(menu)-1:
				break

		print_menu(stdscr, current_row)


curses.wrapper(main)

```